In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2M
import tensorflow_addons as tfa
from sklearn.metrics import classification_report

import joblib
import h5py
import numpy as np

C:\Users\Nans\miniconda3\envs\rakuten_project_tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\Nans\miniconda3\envs\rakuten_project_tf\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, d

In [6]:
# Chemin vers l'objet LabelEncoder sauvegardé
label_encoder_path = "../../models/label_encoder.pkl"
label_encoder = joblib.load(label_encoder_path)
class_labels = label_encoder.classes_

def run_model_combi(X_test_img, y_test):
    model = load_model('../../models/tuned_transfer_learning_effi3b_weightclass.h5')
    pred = model.predict(X_test_img)
    
    y_pred = np.argmax(pred, axis = -1)
    y_true = np.argmax(y_test, axis = -1)
    
    report = classification_report(np.array(y_true), np.array(y_pred), target_names=list(map(str,label_encoder.classes_)), output_dict=True)
    cm = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
    normalized_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    return report, normalized_cm


In [10]:
# Lire le tableau à partir du fichier HDF5
with h5py.File('xy_val_224.h5', 'r') as f:
    x_img_val = f['x_img_val'][:1000]
    y_val = f['y_val'][:1000]

In [11]:
report, cm = run_model_combi(x_img_val, y_val)

32/32 [==============================] - 5s 101ms/step
